### Pix2Pix modeling

Model repo: https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

Paper: https://arxiv.org/pdf/1611.07004.pdf

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from cellvision_lib import train_test_val
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Proprocesing the data for pix2pix model
import os
import glob
from shutil import copyfile

MAX_DEPTH = 100
NUM_SAMPLES = 109

# folder_path = '/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-normalized'
# train, test, val = train_test_val(folder_path, channels = 1, train_pp = .67, test_pp = .165, val_pp = .165, set_seed = 1)

# train[0:10]

def clear_test_files(pix2pix_path):
    outer_paths = ['A','B']
    inner_paths = ['test','train','val']
    for outer in outer_paths:
        for inner in inner_paths:
            print('{root}/{split}/{inner}'.format(root=pix2pix_path, split=outer, inner=inner))
            files = glob.glob('{root}/{split}/{inner}/*'.format(root=pix2pix_path, split=outer, inner=inner))
            for f in files:
                os.remove(f)
                
def setup_images_for_pix2pix():
    pix_folder_path = '/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/testing'
    clear_test_files(pix_folder_path)
    folder_path = '/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-normalized'
    train, test, val = train_test_val(folder_path, 
                                      channels = 1, 
                                      train_pp = .67, 
                                      test_pp = .165, 
                                      val_pp = .165, 
                                      set_seed = 1)
    train_images = train[0:10]
    test_images = test[0:10]
    val_images = val[0:10]
    print(train_images)
    
    for i, (comp, ref) in enumerate(train_images):
        print(i)
        print(comp)
        print(ref)
        new_comp_path = pix_folder_path + '/A/train/{}.img'.format(i)
        new_ref_path = pix_folder_path + '/B/train/{}.img'.format(i)
        copyfile(comp, new_comp_path)
        copyfile(ref, new_ref_path)
    
    for comp, ref in test_images:
        new_comp_path = pix_folder_path + '/A/test/{}.img'.format(i)
        new_ref_path = pix_folder_path + '/B/test/{}.img'.format(i)
        copyfile(comp, new_comp_path)
        copyfile(ref, new_ref_path)
        
    for comp, ref in val_images:
        new_comp_path = pix_folder_path + '/A/val/{}.img'.format(i)
        new_ref_path = pix_folder_path + '/B/val/{}.img'.format(i)
        copyfile(comp, new_comp_path)
        copyfile(ref, new_ref_path)


setup_images_for_pix2pix()
print(glob.glob('/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/testing/A/train'))
print(glob.glob('/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/testing/B/train'))

# print(len(channel1_comps))

/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/testing/A/test
/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/testing/A/train
/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/testing/A/val
/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/testing/B/test
/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/testing/B/train
/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/testing/B/val
[['/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-normalized/sample68_channel1_z93.tif', '/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-normalized/sample68_channel6_z93.tif'], ['/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-normalized/sample57_channel1_z33.tif', '/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-normalized/sample57_channel6_z33.tif'], ['/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-normalized/sample74_channel1_z35.tif', '/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-normalized/sample74_channel6_z3